# Department of Transportation - Oil Spill Mapping

In [ ]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import openpyxl
import folium
import json
import gmaps.datasets
import numpy as np
import re
# from shapely.geometry import Point, Polygon
# import geopandas as gpd
# import descartes
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import geopandas

%matplotlib inline
%matplotlib notebook

# Import API key
from config import g_map

# Configure gmaps
gmaps.configure(api_key=g_map)

## Bring in 2010 - Present Data

In [ ]:
# 1. Read the Data_Columns_and_spills into a DataFrame. All sheets
# Unedited excel files
xlfile_2010 = pd.ExcelFile("../Resources/Data_columns_and_spills.xlsx", engine='openpyxl')
spill2010_df = xlfile_2010.parse('2010-Present') #'2010-Present' is the sheet name in the excel file
#spill2010_df is all data all columns




In [ ]:
spill2010_df

In [ ]:
spills = spill2010_df.to_dict('records')
spills

In [ ]:
# import pandas as pd
# from shapely.geometry import Point
# import geopandas as gpd
# from geopandas import GeoDataFrame

# geometry = [Point(xy) for xy in zip(latlong_df['LOCATION_LATITUDE'], latlong_df['LOCATION_LONGITUDE'])]
# gdf = GeoDataFrame(latlong_df, geometry=geometry)   

# #this is a simple map that goes with geopandas
# world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red', markersize=15);

In [ ]:
# formatting dataframe for mapping display. 
# Have to change datetime to string for json conversion
spill2010_df["REPORT_RECEIVED_DATE"] = spill2010_df["REPORT_RECEIVED_DATE"].dt.strftime('%Y-%m-%d %H:%M:%S')
spill2010_df["LOCAL_DATETIME"] = spill2010_df["LOCAL_DATETIME"].dt.strftime('%Y-%m-%d %H:%M:%S')
spill2010_df["NRC_RPT_DATETIME"] = spill2010_df["NRC_RPT_DATETIME"].dt.strftime('%Y-%m-%d %H:%M:%S')
spill2010_df["SHUTDOWN_DATETIME"] = spill2010_df["SHUTDOWN_DATETIME"].dt.strftime('%Y-%m-%d %H:%M:%S')
spill2010_df["RESTART_DATETIME"] = spill2010_df["RESTART_DATETIME"].dt.strftime('%Y-%m-%d %H:%M:%S')
spill2010_df["INCIDENT_IDENTIFIED_DATETIME"] = spill2010_df["INCIDENT_IDENTIFIED_DATETIME"].dt.strftime('%Y-%m-%d %H:%M:%S')
spill2010_df["ON_SITE_DATETIME"] = spill2010_df["ON_SITE_DATETIME"].dt.strftime('%Y-%m-%d %H:%M:%S')
spill2010_df["PREPARED_DATE"] = spill2010_df["PREPARED_DATE"].dt.strftime('%Y-%m-%d %H:%M:%S')
# Creating additional columns of UNINTENTIONAL_RELEASE_BBLS and PRPTY as a string for popup but still need 
# UNINTENTIONAL_RELEASE_BBLS and PRPTY as float for plotting
spill2010_df["UNINTENTIONAL_RELEASE_BBLS_STR"] = spill2010_df["UNINTENTIONAL_RELEASE_BBLS"].map("{:,.2f}".format)
spill2010_df["PRPTY_STR"] = spill2010_df["PRPTY"].map("${:,.0f}".format)


In [ ]:
spill2010_df["COMPONENT_AGE(YEARS)"] = (spill2010_df["IYEAR"] - spill2010_df["INSTALLATION_YEAR"]) + 1
spill2010_df.head()

## Mapping 2010-Present Spills

In [ ]:
# Pulling useful columns for mapping purposes
my_columns = [
    'REPORT_NUMBER',
    'NAME',
    'LOCATION_LATITUDE',
    'LOCATION_LONGITUDE',
    'COMMODITY_RELEASED_TYPE',
    'UNINTENTIONAL_RELEASE_BBLS',
    'UNINTENTIONAL_RELEASE_BBLS_STR',
    'ON_OFF_SHORE',
    'ONSHORE_CITY_NAME',
    'ONSHORE_COUNTY_NAME',
    'ONSHORE_STATE_ABBREVIATION',
    'SYSTEM_PART_INVOLVED',
    'INSTALLATION_YEAR',
    'COMPONENT_AGE(YEARS)',
    'CAUSE',
    'ACCIDENT_PSIG',
    'MOP_PSIG',
    'PRPTY',
    'PRPTY_STR'
]

In [ ]:
# mapping columns called info2010_df
mapping_df = spill2010_df[my_columns]
# Creating another dataframe of first 100 rows called 'test' t2010_df
# Used to get map functioning and limiting API requests
# t2010_df = info2010_df[:100]
# t2010_df.to_json('test_df.json') # exports DataFrame to json for javascript and mapping functionality
# t2010_df
mapping_df

In [ ]:
# # Join lat and long into one column...might be easier for mapping json
# t2010_df.isnull().sum(axis = 0)

In [ ]:
mapping_df = mapping_df.fillna('N/A')
mapping_df

In [ ]:
# dataframe to dictionary
mapping_dict = mapping_df.to_dict('records')


In [ ]:
# making dict into easy to read json for JS array
json_mapping = json.dumps(mapping_dict, indent = 4)
print(json_mapping)

In [ ]:
mapping_dict = mapping_df.to_dict('records')
json_mapping = json.dumps(mapping_dict, indent = 4)
with open('mapping.json','w') as data:
    data.write(json_mapping)

## Mapping 2010-Present 'info2010_df' spill locations ~4000 locations

In [ ]:
info_box_template = """
<dl>
<dt>Operator ID: </dt><dd>{OPERATOR_ID}</dd>
<dt>City</dt><dd>{ONSHORE_CITY_NAME}</dd>
<dt>State</dt><dd>{ONSHORE_STATE_ABBREVIATION}</dd>
<dt>Spill Commodity</dt><dd>{COMMODITY_RELEASED_TYPE}</dd>
<dt>Spill Volume (bbls)</dt><dd>{UNINTENTIONAL_RELEASE_BBLS}</dd>
</dl>

"""


spill_info = [info_box_template.format(**row) for index, row in info2010_df.iterrows()]

In [ ]:
# Create map from lat and long
locations = info2010_df[["LOCATION_LATITUDE","LOCATION_LONGITUDE"]]
spill = info2010_df["UNINTENTIONAL_RELEASE_BBLS"]
fig = gmaps.figure(center=(36, -97), zoom_level=4)
heat_layer = gmaps.heatmap_layer(locations, weights=spill,dissipating=False,
            max_intensity=30000, point_radius=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=spill_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
#Display the figure
fig

## Add another map 
### Filtering only pipeline, corrosion failures
### showing property damage instead of spill volume

In [ ]:
my_columns = [
    'REPORT_NUMBER',
    'OPERATOR_ID',
    'LOCATION_LATITUDE',
    'LOCATION_LONGITUDE',
    'COMMODITY_RELEASED_TYPE',
    'UNINTENTIONAL_RELEASE_BBLS',
    'ON_OFF_SHORE',
    'ONSHORE_CITY_NAME',
    'ONSHORE_COUNTY_NAME',
    'ONSHORE_STATE_ABBREVIATION',
    'SYSTEM_PART_INVOLVED',
    'INSTALLATION_YEAR',
    'CAUSE',
    'PRPTY'
    ]

In [ ]:
pipeline_df = spill2010_df[my_columns]
pipeline_df

In [ ]:
# # changing formatting of Property column to currency
# pipeline_df['PRPTY'] = pipeline_df['PRPTY'].apply(lambda x: "${:,.0f}".format(x))
# pipeline_df

In [ ]:
# Filtering causes; showing only corrosion failure, material failure of pipe or weld and equipment failure.
pipeline_df.loc[(pipeline_df["CAUSE"] == "CORROSION FAILURE") & 
                       (pipeline_df["CAUSE"] == "MATERIAL FAILURE OF PIPE OR WELD") &
                       (pipeline_df["CAUSE"] == "EQUIPMENT FAILURE")]
pipeline_df

In [ ]:
info_box_template = """
<dl>
<dt>Operator ID: </dt><dd>{OPERATOR_ID}</dd>
<dt>City</dt><dd>{ONSHORE_CITY_NAME}</dd>
<dt>State</dt><dd>{ONSHORE_STATE_ABBREVIATION}</dd>
<dt>Cause:</dt><dd>{CAUSE}</dd>
<dt>Spill Commodity</dt><dd>{COMMODITY_RELEASED_TYPE}</dd>
<dt>Spill Volume (bbls)</dt><dd>{UNINTENTIONAL_RELEASE_BBLS}</dd>
<dt>Spill Cost</dt><dd>{PRPTY}</dd>
</dl>

"""

# Using the pipeline_df to gather spill info for marker data...
spill_info = [info_box_template.format(**row) for index, row in pipeline_df.iterrows()]

In [ ]:
# Create map from lat and long
locations = pipeline_df[["LOCATION_LATITUDE","LOCATION_LONGITUDE"]]
spill = pipeline_df["PRPTY"]
fig = gmaps.figure(center=(36, -97), zoom_level=4)
heat_layer = gmaps.heatmap_layer(locations, weights=spill,dissipating=False,
            max_intensity=1000000000, point_radius=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=spill_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
#Display the figure
fig